# Multi Resolution Simulation

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common
from gpuocean.SWEsimulators import CDKLM16, CDKLM16pair

ImportError: cannot import name 'CDKLM16pair' from 'gpuocean.SWEsimulators' (/home/florianb/havvarsel/gpuocean/src/gpuocean/SWEsimulators/__init__.py)

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
%cuda_context_handler gpu_ctx0

In [ ]:
%cuda_context_handler gpu_ctx1

In [ ]:
%cuda_context_handler gpu_ctx2

In [ ]:
%cuda_context_handler gpu_ctx3

In [ ]:
%cuda_context_handler gpu_ctx4

## Setting-up case 

In [ ]:
def initBump(data_args, dataShape, d_shift=1e6,D=0.5*1e6):

    eta0  = np.zeros(dataShape, dtype=np.float32, order='C')
    hu0   = np.zeros(dataShape, dtype=np.float32, order='C')
    hv0   = np.zeros(dataShape, dtype=np.float32, order='C')

    x_center = data_args["dx"]*(data_args["nx"]+4)*0.5
    y_center = data_args["dy"]*(data_args["ny"]+4)*0.5

    scale = 1e9
    for j in range(data_args["ny"] + 4):
        for i in range(data_args["nx"] + 4):
            x = data_args["dx"]*i - x_center
            y = data_args["dy"]*j - y_center

            d = np.sqrt(x**2 + y**2)
            
            eta0[j, i] += 0.1*(1.0+np.tanh(-(d-d_shift)/D))

    return eta0, hu0, hv0

In [ ]:
def initLevel(l):
    data_args = {}
    data_args["nx"] = 2**l - 4
    data_args["ny"] = 2**l - 4 
    dataShape = (data_args["ny"] + 4, data_args["nx"] + 4)

    data_args["dx"] = 2**(19-l)*100
    data_args["dy"] = 2**(19-l)*100

    data_args["dt"] = 0.0
    data_args["g"] = 9.81
    data_args["f"] = 1.2e-4
    data_args["r"] = 0.0
    data_args["boundary_conditions"] = Common.BoundaryConditions(1,1,1,1)

    data_args["eta0"], data_args["hu0"], data_args["hv0"] = initBump(data_args, dataShape)
    
    H0 = 1000.0
    data_args["H"] = np.ma.array(np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*H0, mask=False)

    return data_args

In [ ]:
# fig, axs = plt.subplots(1,3, figsize=(20,10))

# for i in range(3):
#     data_args = initLevel(10+i)
#     im = axs[i].imshow(data_args["eta0"])
#     plt.colorbar(im, ax=axs[i], shrink=0.5)

In [ ]:
def runLevel(gpu_ctx, l, pert=False, T=250000):
    if isinstance(l, dict):
        data_args = l 
    else:
        data_args = initLevel(l)
    sim = CDKLM16.CDKLM16(gpu_ctx, **data_args)
    if pert:
        pert_args = {
            "small_scale_perturbation" : True,
            "small_scale_perturbation_amplitude": 1e-3,
            "small_scale_perturbation_interpolation_factor" : 31,
            "small_scale_perturbation_dx" : 2**7 *100,
            "small_scale_perturbation_dy" : 2**7 *100,
            }
        sim_pair = CDKLM16pair.CDKLM16pair(sim, **pert_args)
        sim_pair.perturbSimPair()
        sim_pair.step(T)
        return sim_pair.sim
    else:
        sim.step(T)
        return sim 

In [ ]:
sim0 = runLevel(gpu_ctx0, 7)

In [ ]:
sim1 = runLevel(gpu_ctx1, 8)

In [ ]:
sim2 = runLevel(gpu_ctx2, 9)

In [ ]:
sim3 = runLevel(gpu_ctx3, 10)

In [ ]:
sim4 = runLevel(gpu_ctx4, 11)

In [ ]:
fig, axs = plt.subplots(5,3, figsize=(15,15))

for j in range(5):
    for i in range(3):
        im = axs[j,i].imshow(eval("sim"+str(j)).download()[i])
        plt.colorbar(im, ax=axs[j,i], shrink=0.5)
plt.tight_layout()

In [ ]:
ls = [7, 8, 9, 10, 11]

In [ ]:
N_var = 2

est_vars = np.zeros((len(ls), 3))

for l_idx, l in enumerate(ls):
    print("Level ", l_idx)
    data_args = initLevel(l)
    est_mean = np.zeros((3,data_args["ny"],data_args["nx"]))

    samples = np.zeros((3,data_args["ny"],data_args["nx"],N_var))    
    for i in range(N_var):
        print("Sample ", i)
        sim = runLevel(gpu_ctx, data_args, pert=True)
        eta, hu, hv = sim.download(interior_domain_only=True)
        samples[0,:,:,i] = eta
        samples[1,:,:,i] = hu
        samples[2,:,:,i] = hv
        sim.cleanUp()
    est_vars[l,0] = np.sqrt(np.average(np.var(samples[0], axis=-1)**2))
    est_vars[l,1] = np.sqrt(np.average(np.var(samples[1], axis=-1)**2))
    est_vars[l,2] = np.sqrt(np.average(np.var(samples[2], axis=-1)**2))